In [13]:
import brickschema

g = brickschema.Graph()
g.load_file('chamber_shacl_expanded.ttl')

<Graph identifier=N5225c3956b674a44860f93aa3fdf5894 (<class 'brickschema.graph.Graph'>)>

In [21]:
# Query test 1
query1 = g.query(
    """SELECT ?ahu ?damper ?damper_pos WHERE {
    ?ahu a brick:Air_Handling_Unit .
    ?ahu brick:hasPart ?damper .
    ?damper a brick:Damper .
    ?damper brick:feeds ?cc .
    ?cc a brick:Chilled_Water_Coil .
    ?damper_pos a brick:Damper_Position_Sensor .
    ?damper brick:hasPoint ?damper_pos
}"""
)
print("\nQuery test 1 results:")
for row in query1:
    print(row)


Query test 1 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_dmpr_pos_ao_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_supply'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#supply_damper_pos_10'))


In [23]:
# Query test 2
query2 = g.query(
    """SELECT * WHERE{ 
    ?sensor rdf:type/rdfs:subClassOf* brick:Setpoint .
    ?sensor brick:isPointOf ?equipment
}"""
)
print("\nQuery test 2 results:")
for row in query2:
    print(row)


Query test 2 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dsp_fixed_setpoint_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#press_stpt_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dsp_fixed_setpoint_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#press_stpt_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dat_sp_clg_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dat_sp_clg_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#WH_1'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#HWST_stp'))
(rdflib.te

In [16]:
# Query test 3
query3 = g.query(
    """SELECT ?position WHERE{
    ?ahu a brick:Air_Handling_Unit .
    ?position a brick:Damper_Position_Sensor .
    ?damper a brick:Damper .
    ?zone a brick:HVAC_Zone .
    ?ahu brick:hasPart ?damper .
    ?damper brick:hasPoint ?position .
    ?damper brick:feeds ?zone
    }
    """
)
print("\nQuery test 3 results:")
for row in query3:
    print(row)


Query test 3 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_dmpr_pos_ao_6'),)


In [17]:
# Query test 4
query4 = g.query(
    """SELECT ?sat WHERE{
        ?ahu a brick:Air_Handling_Unit .
        ?sat a brick:Supply_Air_Temperature_Sensor .
        ?ahu brick:hasPoint ?sat
    }
    """
)
print("\nQuery test 4 results:")
for row in query4:
    print(row)


Query test 4 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#sa_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#coil_da_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#eff_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_sa_temp_6'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#overhead_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#underfloor_sa_temp_10'),)
